In [2]:

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D


2022-09-12 00:13:21.590003: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sahitya/.local/lib/python3.9/site-packages/cv2/../../lib64:
2022-09-12 00:13:21.590034: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
train_data_dir='/home/sahitya/Desktop/Projects/Age_gender_detection/fer2013/train'
validation_data_dir='/home/sahitya/Desktop/Projects/Age_gender_detection/fer2013/validation'

In [4]:
num_classes=7
img_rows,img_cols=48,48
batch_size=16

In [5]:
train_datagen=ImageDataGenerator(
                            rescale=1./255,
rotation_range=30,
shear_range=0.3,
zoom_range=0.3,
width_shift_range=0.4,
height_shift_range=0.4,
horizontal_flip=0.4,
fill_mode='nearest')

In [6]:
train_datagen=ImageDataGenerator(rescale=1./255)
train_generator=train_datagen.flow_from_directory(
train_data_dir,
color_mode='grayscale',
target_size=(img_rows,img_cols),
batch_size=batch_size,
class_mode='categorical',
shuffle=True)

Found 28709 images belonging to 7 classes.


In [7]:
validation_datagen=ImageDataGenerator(rescale=1./255)
validation_generator=validation_datagen.flow_from_directory(
train_data_dir,
color_mode='grayscale',
target_size=(img_rows,img_cols),
batch_size=batch_size,
class_mode='categorical',
shuffle=True)

Found 28709 images belonging to 7 classes.


In [8]:
import tensorflow
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.core import Activation,Flatten,Dropout,Dense
from tensorflow.keras.optimizers import RMSprop ,SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [54]:
model=Sequential()
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 activation (Activation)     (None, 48, 48, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 48, 48, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                        

In [15]:
from tensorflow.keras.optimizers import RMSprop ,SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [60]:
checkpoint=ModelCheckpoint('./saved_models',
                              monitor='val_loss',
                              mode='min',
                              save_best_only=True,
                              verbose=1)
earlystop=EarlyStopping(monitor='val_loss',
                           min_delta=0,
                           patience=3,
                           verbose=1,
                           restore_best_weights=True)
reduce_lr=ReduceLROnPlateau(montor='val_loss',
                               factor=0.2,
                               patience=3,
                               verbose=1,
                               min_delata=0.001)
call_backs=[earlystop,checkpoint,reduce_lr]

model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=0.01),
    metrics=['accuracy'])

nb_train_samples=28273
nb_validation_samples=3534
epochs=10
history=model.fit_generator(
train_generator,
steps_per_epoch=nb_train_samples//batch_size,
epochs=epochs,
callbacks=call_backs,
validation_data=validation_generator,
validation_steps=nb_validation_samples//batch_size)

/home/sahitya/.local/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/tmp/ipykernel_43783/2172657668.py:26: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(


Epoch 1/10
1767/1767 [==============================] - ETA: 0s - loss: 1.3990 - accuracy: 0.4693
Epoch 00001: val_loss improved from inf to 1.32257, saving model to ./saved_models
INFO:tensorflow:Assets written to: ./saved_models/assets
1767/1767 [==============================] - 514s 290ms/step - loss: 1.3990 - accuracy: 0.4693 - val_loss: 1.3226 - val_accuracy: 0.5006 - lr: 0.0100
Epoch 2/10
1767/1767 [==============================] - ETA: 0s - loss: 1.3186 - accuracy: 0.5027
Epoch 00002: val_loss improved from 1.32257 to 1.14233, saving model to ./saved_models
INFO:tensorflow:Assets written to: ./saved_models/assets
1767/1767 [==============================] - 705s 399ms/step - loss: 1.3186 - accuracy: 0.5027 - val_loss: 1.1423 - val_accuracy: 0.5699 - lr: 0.0100
Epoch 3/10
1767/1767 [==============================] - ETA: 0s - loss: 1.2697 - accuracy: 0.5277
Epoch 00003: val_loss did not improve from 1.14233
1767/1767 [==============================] - 509s 288ms/step - loss: 1.

In [9]:
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import classification_report ,confusion_matrix
import numpy as np
bn_train_samples=28273
nb_validation_samples=3534
validation_generator=validation_datagen.flow_from_directory(
train_data_dir,
color_mode='grayscale',
target_size=(img_rows,img_cols),
batch_size=batch_size,
class_mode='categorical',
shuffle=True)
class_labels=validation_generator.class_indices
class_labels={v:k for k,v in class_labels.items()}
classes=list(class_labels.values())

y_pred=model.predict(validation_generator,nb_validation_samples//batch_size)
y_pred=np.argmax(y_pred,axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes,y_pred))
print('Confusion Report')
target_names=list(class_labels.values())
print(classification_report(validation_generator.classes,y_pred,target_names=target_names))
cnf_matrix=confusion_matrix(validation_generator.classes,y_pred)
plt.imshow(cnf_matrix,interpolation='nearest')
plt.colorbar()
tick_marks=np.arange(len(classes))
plt.xticks(tick_marks,classes,rotation=90)
plt.yticks(tick_marks,classes)
plt.show()

Found 28709 images belonging to 7 classes.


NameError: name 'model' is not defined

In [10]:
import os
from keras.models import load_model

In [11]:
import numpy as np

In [12]:
classifier= keras.models.load_model("/home/sahitya/Desktop/saved_models/")


2022-09-12 00:13:46.915278: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sahitya/.local/lib/python3.9/site-packages/cv2/../../lib64:
2022-09-12 00:13:46.915314: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-12 00:13:46.915345: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sahitya-Inspiron-15-3567): /proc/driver/nvidia/version does not exist
2022-09-12 00:13:46.915564: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
validation_generator=validation_datagen.flow_from_directory(
train_data_dir,
color_mode='grayscale',
target_size=(img_rows,img_cols),
batch_size=batch_size,
class_mode='categorical',
shuffle=True)
class_labels=validation_generator.class_indices
class_labels={v:k for k,v in class_labels.items()}
classes=list(class_labels.values())
print(class_labels)

Found 28709 images belonging to 7 classes.
{0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}


In [13]:
from keras.models import load_model
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile,join
import re

def draw_test(name,pred,im,true_label):
    BLACK=[0,0,0]
    expanded_image=cv2.copyMakeBorder(im,160,0,0,300,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image,'predicted'+pred,(120,60),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    cv2.putText(expanded_image,'true'+true_label,(20,120),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow(name,expanded_image)

In [1]:
def getRandomImage(path,img_width,img_height):
    folders=list(filter(lambda x:os.path.isdir(os.path.join(path,x)),os.listdir(path)))
    random_directory=np.random.randint(0,len(folders))
    path_class=folders[random_directory]
    file_path=path+path_class
    file_names=[f for f in listdir(file_path) if isfile(join(file_path,f))]
    random_file_index=np.random.randint(0,len(file_names))
    image_name=file_names[random_file_index]
    final_path=file_path+'/'+image_name
    return image.load_img(final_path,target_size=(img_width,img_height),grayscale=True),final_path,path_class

     

In [15]:
from keras.preprocessing import image
img_width,img_height=48,48
classifier.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=0.001),
             metrics=['accuracy'])

files=[]
prediction=[]
true_labels=[]

for i in range(0,10):
    path='/home/sahitya/Desktop/Projects/Age_gender_detection/fer2013/validation/'
    img,final_path,true_label=getRandomImage(path,img_width,img_height)
    files.append(final_path)
    true_labels.append(true_label)
    x=image.img_to_array(img)
    x=x*1.0/255
    x=np.expand_dims(x,axis=0)
    images=np.vstack([x])
    pred=classifier.predict(images,batch_size=10)
    classes=np.argmax(pred,axis=1)
    print(classes)
    prediction.append(classes)
    
    


/home/sahitya/.local/lib/python3.9/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)
/home/sahitya/.local/lib/python3.9/site-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


[2]
[3]
[2]
[4]
[3]
[0]
[0]
[0]
[5]
[5]


In [ ]:
for i in range(0,len(files)):
    image=cv2.imread((files[i]))
    image=cv2.resize(image,None,fx=3,fy=3,interpolation=cv2.INTER_CUBIC)
    draw_test('Prediction',class_labels[prediction[i][0]],image,true_labels[i])
    cv2.waitKey(0)
cv2.destroyAllWindows(0)
    

'/home/sahitya/Desktop'

In [16]:
face_classifier=cv2.CascadeClassifier('/home/sahitya/Desktop/haarcascade_frontalface_default.xml')
def face_detector(img):
    gray=cv2.cvtColor(img.copy(),cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiSacale(gray,1.3,5)
    if faces is ():
        return (0,0,0,0),np.zeros((48,48),np.uint8),img
    allfaces=[]
    rects=[]
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+w),(255,0,0),2)
        roi_gray=gray[y:y+h,x:x+w]
        roi_gray=cv2.resize(roi_gray,(48,48),interpolation=ccv2.INTER_AREA)
        allfaces.append(roi_gray)
        rects.apppend((x,w,y,h))
    return rects,allfaces,img

i=0 
for face in faces:
    roi=face.astype('float')/255.0
    roi=img_to_array(roi)
    roi=np.expand_dims(roi,axis=0)
    preds=model.predict(roi)[0]
    label=class_label[preds.argmax()]
    label_position=(rects[i][0]+=int((rects[1][i]/2)) ,abs(rects[i][2]))
    i+=1
    cv2.putText(image,label,label_position,cv2.FRONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
cv2.imshow('Emotion Detector',image)
cv2.waitKey(0)
cv2.DestroyAllWindows()
    
    
    

SyntaxError: invalid syntax (3180759899.py, line 24)

In [16]:
import cv2
import numpy as np
from time import sleep
from keras.preprocessing.image import img_to_array
face_classifier=cv2.CascadeClassifier('/home/sahitya/Desktop/haarcascade_frontalface_default.xml')
def face_detector(img):
    gray=cv2.cvtColor(img.copy(),cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    if faces is ():
        return (0,0,0,0),np.zeros((48,48),np.uint8),img
    allfaces=[]
    rects=[]
    for (x,y,w,h) in faces:
        x=x-50
        w=w+50
        y=y-50
        h=h-50
        cv2.rectangle(img,(x,y),(x+w,y+w),(255,0,0),2)
        roi_gray=gray[y:y+h,x:x+w]
    try:
        roi_gray=cv2.resize(roi_gray,(48,48),interpolation=ccv2.INTER_AREA)
    except: 
        return (x,w,y,h),np.zeros((48,48),np.uint8),img
    return (x,y,w,h),roi_gray,img


<>:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_30267/2209127122.py:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In [18]:
import cv2
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    rect,face,image=face_detector(frame)
    if np.sum([face])!=0.0:
        roi=face.astype('float')/255.0
        roi=img_to_array(roi)
        
        roi=np.expand_dims(roi,axis=0)
        
        preds=classfier.predict((roi)[0])
        
        label=class_labels[preds.argmax()]
        
        label_position=(rect[0]+int((rect[1]/2)),rect[2]+25)
        cv2.putText(image,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    else:
        cv2.putText(image,'No Face Found',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    
    cv2.imshow('All',image)
    if cv2.waitKey(1)==13:
        break
        
cap.release()
cv2.destroyAllWindows()